<a href="https://colab.research.google.com/github/cseybold/Weather/blob/main/Weather.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install meteostat

In [3]:
pip install geopy

In [4]:
!pip install requests

In [5]:
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from meteostat import Point, Daily, Stations
import pandas as pd
import xarray as xr
from geopy.distance import geodesic
import requests

In [6]:
import ee
from google.colab import auth
auth.authenticate_user(project_id='weatherproj-427900')

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='weatherproj-427900')

In [7]:
firedf = pd.read_csv('/content/drive/MyDrive/WFIGS_inc_loc.csv')

firedf = firedf[['IncidentSize', 'FireDiscoveryDateTime', 'IncidentTypeCategory', 'InitialLatitude', 'InitialLongitude', 'POOState']]
firedf['FireDiscoveryDateTime'] = pd.to_datetime(firedf['FireDiscoveryDateTime'])
firedf.dropna(subset=['InitialLatitude', 'InitialLongitude'], inplace=True)

cafire = firedf[firedf['POOState'] == 'US-CA']
cafire = cafire[cafire['IncidentTypeCategory'] == 'WF']
cafire = cafire[cafire['IncidentSize'] > 1]
cafire.tail()

,IncidentSize,FireDiscoveryDateTime,IncidentTypeCategory,InitialLatitude,InitialLongitude,POOState
309485,1.80,2024-06-25 21:15:15,WF,35.561729,-120.516991,US-CA
309497,1.25,2024-06-25 18:04:39,WF,41.759312,-122.490208,US-CA
309520,11.03,2024-06-25 23:30:19,WF,38.845861,-123.019703,US-CA
309569,12.00,2024-06-26 02:28:04,WF,36.631882,-119.322600,US-CA
309592,53.91,2024-06-26 05:00:46,WF,37.732422,-120.596351,US-CA


In [8]:
cafire = cafire.reset_index(drop=True)
cafire.tail()

,IncidentSize,FireDiscoveryDateTime,IncidentTypeCategory,InitialLatitude,InitialLongitude,POOState
3811,1.80,2024-06-25 21:15:15,WF,35.561729,-120.516991,US-CA
3812,1.25,2024-06-25 18:04:39,WF,41.759312,-122.490208,US-CA
3813,11.03,2024-06-25 23:30:19,WF,38.845861,-123.019703,US-CA
3814,12.00,2024-06-26 02:28:04,WF,36.631882,-119.322600,US-CA
3815,53.91,2024-06-26 05:00:46,WF,37.732422,-120.596351,US-CA


In [9]:
# Create a Point for a sample location
location = Point(37.7749, -122.4194) # Example: San Francisco

# Get daily data for a specific day
start_time = datetime(2023, 11, 10)
end_time = start_time + timedelta(days=1)
data = Daily(location, start_time, end_time)
data = data.fetch()

# Print the available columns (weather stats)
print(data.columns)

Index(['tavg', 'tmin', 'tmax', 'prcp', 'snow', 'wdir', 'wspd', 'wpgt', 'pres',
       'tsun'],
      dtype='object')


In [10]:
testdf = cafire.head(50)

def weather_stats(lat, lon, time):
    # Create a Point for the location
    location = Point(lat, lon)

    # Get hourly data
    data = Daily(location, time, time + timedelta(days=1))
    data = data.fetch()

    # Check if data is empty
    if data.empty:
        # Find the nearest station with data
        stations = Stations()
        stations = stations.nearby(lat, lon)
        station = stations.fetch(1)

        if not station.empty:
            location = Point(station.iloc[0]['latitude'], station.iloc[0]['longitude'])
            data = Daily(location, time, time + timedelta(days=1))
            data = data.fetch()

    # Return the temperature, precipitation, wind speed, and pressure
    if not data.empty:
        return [data.iloc[0]['tavg'], data.iloc[0]['prcp'], data.iloc[0]['wspd'], data.iloc[0]['pres']]
    else:
        return None

# Apply the function to each row and handle None values
def apply_weather_stats(row, i):
    stats = weather_stats(row['InitialLatitude'], row['InitialLongitude'], row['FireDiscoveryDateTime'])
    return stats[i] if stats is not None else None

testdf['Temperature'] = testdf.apply(lambda row: apply_weather_stats(row, 0), axis=1)
testdf['Precipitation'] = testdf.apply(lambda row: apply_weather_stats(row, 1), axis=1)
testdf['WindSpeed'] = testdf.apply(lambda row: apply_weather_stats(row, 2), axis=1)
testdf['Pressure'] = testdf.apply(lambda row: apply_weather_stats(row, 3), axis=1)

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user

In [11]:
testdf['Precipitation'].fillna(0, inplace=True)
testdf.dropna(subset=['Temperature'], inplace=True)
testdf.dropna(subset=['WindSpeed'], inplace=True)
testdf.dropna(subset=['Pressure'], inplace=True)

testdf.tail()

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,IncidentSize,FireDiscoveryDateTime,IncidentTypeCategory,InitialLatitude,InitialLongitude,POOState,Temperature,Precipitation,WindSpeed,Pressure
42,2.85,2019-10-07 08:12:00,WF,33.773090,-116.545600,US-CA,29.3,0.0,10.3,1007.9
44,1.50,2019-09-05 23:17:55,WF,34.448900,-118.056300,US-CA,29.6,0.0,15.8,1015.3
45,80.00,2022-06-21 16:48:36,WF,33.546326,-116.043868,US-CA,32.9,0.0,12.4,1009.5
48,350.00,2019-10-10 19:56:55,WF,33.984818,-117.217944,US-CA,19.9,0.0,13.3,1013.0
49,2.00,2017-07-26 00:46:09,WF,41.675830,-123.420700,US-CA,13.1,0.0,11.3,1018.8


In [12]:
def get_vegetation_index(longitude, latitude, date):
    # Define the dataset
    dataset = ee.ImageCollection('NOAA/VIIRS/001/VNP13A1')

    # Filter the dataset by date and location
    filtered_dataset = dataset.filterDate(date, date + timedelta(days = 40)).filterBounds(ee.Geometry.Point([longitude, latitude]))

    # Select 'EVI' and calculate the mean
    mean_evi = filtered_dataset.select('EVI2').mean()

    # Define a region of interest
    roi = ee.Geometry.Rectangle([longitude-0.01, latitude-0.01, longitude+0.01, latitude+0.01])

    # Reduce the image over the region of interest
    mean_evi = mean_evi.reduceRegion(reducer=ee.Reducer.mean(), geometry=roi, scale=30)

    # Check if 'EVI2' exists in the dictionary
    if 'EVI2' in mean_evi.getInfo():
        mean_evi_value = mean_evi.get('EVI2').getInfo()
    else:
        # Find the nearest station with data
        stations = Stations()
        stations = stations.nearby(latitude, longitude)
        station = stations.fetch(1)

        if not station.empty:
            nearby_lon = station.iloc[0]['longitude']
            nearby_lat = station.iloc[0]['latitude']
            nearby_roi = ee.Geometry.Rectangle([nearby_lon-0.01, nearby_lat-0.01, nearby_lon+0.01, nearby_lat+0.01])
            mean_evi = mean_evi.reduceRegion(reducer=ee.Reducer.mean(), geometry=nearby_roi, scale=30)
            if 'EVI2' in mean_evi.getInfo():
                mean_evi_value = mean_evi.get('EVI2').getInfo()
            else:
                mean_evi_value = None  # No data found even at the nearest station
        else:
            mean_evi_value = None # No nearby stations found

    return mean_evi_value

testdf['VegetationIndex'] = testdf.apply(lambda row: get_vegetation_index(row['InitialLongitude'], row['InitialLatitude'], row['FireDiscoveryDateTime']), axis=1)



Attention required for NOAA/VIIRS/001/VNP13A1! You are using a deprecated asset.
To ensure continued functionality, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/NOAA_VIIRS_001_VNP13A1

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [13]:
def get_mfri(longitude, latitude):
    # Define the dataset
    dataset = ee.ImageCollection('LANDFIRE/Fire/MFRI/v1_2_0')

    # Filter the dataset by location
    filtered_dataset = dataset.filterBounds(ee.Geometry.Point([longitude, latitude]))

    # Check if the image collection is empty
    if filtered_dataset.size().getInfo() == 0:
        return None

    # Select 'MFRI' and calculate the mean
    mean_mfri = filtered_dataset.select('MFRI').mean()

    # Define a region of interest
    roi = ee.Geometry.Rectangle([longitude-0.01, latitude-0.01, longitude+0.01, latitude+0.01])

    # Reduce the image over the region of interest
    mean_mfri = mean_mfri.reduceRegion(reducer=ee.Reducer.mean(), geometry=roi, scale=30)

    # Get the mean MFRI value
    if 'MFRI' in mean_mfri.getInfo():
        mean_mfri_value = mean_mfri.get('MFRI').getInfo()
    else:
        # Find the nearest station with data (assuming you have a way to define 'stations')
        stations = Stations()  # Replace with your station data source
        stations = stations.nearby(latitude, longitude)
        station = stations.fetch(1)

        if not station.empty:
            nearby_lon = station.iloc[0]['longitude']
            nearby_lat = station.iloc[0]['latitude']
            nearby_roi = ee.Geometry.Rectangle([nearby_lon-0.01, nearby_lat-0.01, nearby_lon+0.01, nearby_lat+0.01])
            mean_mfri = mean_mfri.reduceRegion(reducer=ee.Reducer.mean(), geometry=nearby_roi, scale=30)
            if 'MFRI' in mean_mfri.getInfo():
                mean_mfri_value = mean_mfri.get('MFRI').getInfo()
            else:
                mean_mfri_value = None  # No data found even at the nearest station
        else:
            mean_mfri_value = None  # No nearby stations found

    return mean_mfri_value

testdf['MFRI'] = testdf.apply(lambda row: get_mfri(row['InitialLongitude'], row['InitialLatitude']), axis=1)


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [14]:
testdf.dropna(subset=['VegetationIndex'], inplace=True)
testdf.dropna(subset=['MFRI'], inplace=True)
testdf = testdf.where(testdf['MFRI'] > 0)
testdf = testdf.reset_index(drop=True)
testdf['TimeSinceFire'] = ((datetime.now() - testdf['FireDiscoveryDateTime']).dt.days)/365
testdf.tail()

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,IncidentSize,FireDiscoveryDateTime,IncidentTypeCategory,InitialLatitude,InitialLongitude,POOState,Temperature,Precipitation,WindSpeed,Pressure,VegetationIndex,MFRI,TimeSinceFire
36,2.85,2019-10-07 08:12:00,WF,33.773090,-116.545600,US-CA,29.3,0.0,10.3,1007.9,1652.521132,56.128563,4.901370
37,1.50,2019-09-05 23:17:55,WF,34.448900,-118.056300,US-CA,29.6,0.0,15.8,1015.3,1634.994863,12.481210,4.989041
38,80.00,2022-06-21 16:48:36,WF,33.546326,-116.043868,US-CA,32.9,0.0,12.4,1009.5,1651.378559,30.118662,2.194521
39,350.00,2019-10-10 19:56:55,WF,33.984818,-117.217944,US-CA,19.9,0.0,13.3,1013.0,1459.243946,10.675924,4.893151
40,2.00,2017-07-26 00:46:09,WF,41.675830,-123.420700,US-CA,13.1,0.0,11.3,1018.8,3755.749510,3.719308,7.101370


In [15]:
mtemp = testdf['Temperature'].mean()
mprec = testdf['Precipitation'].mean()
mwind = testdf['WindSpeed'].mean()
mveg = testdf['VegetationIndex'].mean()
mMFRI = testdf['MFRI'].mean()
mtimesince = testdf['TimeSinceFire'].mean()
mpressure = testdf['Pressure'].mean()

sdtemp = testdf['Temperature'].std()
sdprec = testdf['Precipitation'].std()
sdwind = testdf['WindSpeed'].std()
sdveg = testdf['VegetationIndex'].std()
sdMFRI = testdf['MFRI'].std()
sdtimesince = testdf['TimeSinceFire'].std()
sdpressure = testdf['Pressure'].std()

testdata = pd.DataFrame({
    'Temperature': [mtemp, sdtemp],
    'Precipitation': [mprec, sdprec],
    'WindSpeed': [mwind, sdwind],
    'VegetationIndex': [mveg, sdveg],
    'MFRI': [mMFRI, sdMFRI],
    'TimeSinceFire': [mtimesince, sdtimesince],
    'Pressure': [mpressure, sdpressure]
}, index=['mean', 'std'])

testdata.head()

,Temperature,Precipitation,WindSpeed,VegetationIndex,MFRI,TimeSinceFire,Pressure
mean,21.605000,0.0,11.707500,1956.778032,16.866271,4.671233,1014.030000
std,6.109469,0.0,6.819632,616.287534,15.995156,1.662524,4.225575


In [16]:
# Apply the function to each row and handle None values
def apply_weather_stats2(row, i):
    stats = weather_stats(row['Latitude'], row['Longitude'], row['Date'])
    return stats[i] if stats is not None else None

def get_time_since_closest_fire(latitude, longitude, firedf):
    """
    Calculates time since the closest fire in firedf for a given latitude and longitude.
    """
    min_distance = float('inf')
    closest_fire_date = None

    for _, fire_row in firedf.iterrows():
        # Validate latitude before calculating distance
        if not (-90 <= fire_row['InitialLatitude'] <= 90):
            print(f"Invalid latitude in firedf: {fire_row['InitialLatitude']}")
            continue  # Skip this row

        # Calculate distance using geodesic (more accurate for lat/long)
        distance = geodesic((latitude, longitude),
                           (fire_row['InitialLatitude'], fire_row['InitialLongitude'])).km

        if distance < min_distance:
            min_distance = distance
            closest_fire_date = fire_row['FireDiscoveryDateTime']

    if closest_fire_date is not None:
        time_since = (datetime.now() - closest_fire_date).days / 365
    else:
        time_since = None  # Handle cases where no fire is found nearby

    return time_since

In [17]:
# Define the data for each column
testdata2 = pd.DataFrame({
    'Temperature': [None],
    'Precipitation': [None],
    'WindSpeed': [None],
    'VegetationIndex': [None],
    'MFRI': [None],
    'TimeSinceFire': [None],
    'Pressure': [None]
})

# Collect user input
latitude = float(input("Enter latitude: "))
longitude = float(input("Enter longitude: "))
date_str = input("Enter date (YYYY-MM-DD): ")
date = pd.to_datetime(date_str)

Enter latitude: 30
Enter longitude: -30
Enter date (YYYY-MM-DD): 20220404


In [18]:
# Apply functions using the user input
temperature = apply_weather_stats2({'Latitude': latitude, 'Longitude': longitude, 'Date': date}, 0)
precipitation = apply_weather_stats2({'Latitude': latitude, 'Longitude': longitude, 'Date': date}, 1)
wind_speed = apply_weather_stats2({'Latitude': latitude, 'Longitude': longitude, 'Date': date}, 2)
pressure = apply_weather_stats2({'Latitude': latitude, 'Longitude': longitude, 'Date': date}, 3)
vegetation_index = get_vegetation_index(longitude, latitude, date)
mfri = get_mfri(longitude, latitude)
time_since_fire = get_time_since_closest_fire(latitude, longitude, firedf)

if mfri is None:
    mfri = testdata.loc['mean', 'MFRI']

# Create a new row for testdata
new_row = [temperature, precipitation, wind_speed, pressure, vegetation_index, mfri, time_since_fire]
testdata.loc['new_data'] = new_row

testdata.head()

Invalid latitude in firedf: -1764.78333333
Invalid latitude in firedf: 3549.602
Invalid latitude in firedf: 100.188333
Invalid latitude in firedf: 119.0
Invalid latitude in firedf: 94.966


,Temperature,Precipitation,WindSpeed,VegetationIndex,MFRI,TimeSinceFire,Pressure
mean,21.605000,0.0,11.707500,1956.778032,16.866271,4.671233,1014.030000
std,6.109469,0.0,6.819632,616.287534,15.995156,1.662524,4.225575
new_data,14.900000,0.0,17.900000,1023.600000,NaN,16.866271,8.553425


In [19]:
# Create an empty Series to store the results
result = pd.Series(index=testdata.columns)

# Iterate over each column in the DataFrame
for column in testdata.columns:
    if testdata.loc['std', column] == 0:
        value = 0
    else:
        value = (testdata.loc['new_data', column] - testdata.loc['mean', column]) / testdata.loc['std', column]

    # Add the result to the new DataFrame
    result.loc[column] = value

# Print the result
testdata.loc['stdaway'] = result

testdata.head()

,Temperature,Precipitation,WindSpeed,VegetationIndex,MFRI,TimeSinceFire,Pressure
mean,21.605000,0.0,11.707500,1956.778032,16.866271,4.671233,1014.030000
std,6.109469,0.0,6.819632,616.287534,15.995156,1.662524,4.225575
new_data,14.900000,0.0,17.900000,1023.600000,NaN,16.866271,8.553425
stdaway,-1.097477,0.0,0.908040,-1.514193,NaN,7.335254,-237.950217


In [20]:
import math

fire_danger_1 = 2*testdata.loc['stdaway', 'Temperature'] - testdata.loc['stdaway', 'Precipitation'] \
                + testdata.loc['stdaway', 'WindSpeed'] + testdata.loc['stdaway', 'VegetationIndex'] \
                - testdata.loc['stdaway', 'MFRI']/3
fire_danger = 10/(1+math.exp(-1*fire_danger_1))
print(fire_danger)

nan


In [27]:
weatherClass = pd.read_csv('/content/drive/MyDrive/weather_classification_data.csv')
weatherClass.head()

,Temperature,Humidity,Wind Speed,Precipitation (%),Cloud Cover,Atmospheric Pressure,UV Index,Season,Visibility (km),Location,Weather Type
0,14.0,73,9.5,82.0,partly cloudy,1010.82,2,Winter,3.5,inland,Rainy
1,39.0,96,8.5,71.0,partly cloudy,1011.43,7,Spring,10.0,inland,Cloudy
2,30.0,64,7.0,16.0,clear,1018.72,5,Spring,5.5,mountain,Sunny
3,38.0,83,1.5,82.0,clear,1026.25,7,Spring,1.0,coastal,Sunny
4,27.0,74,17.0,66.0,overcast,990.67,1,Winter,2.5,mountain,Rainy


In [32]:
weatherClass.rename(columns={'Precipitation (%)': 'Precipitation', 'Atmospheric Pressure': 'Pressure'}, inplace=True)
weatherClass.columns = [col.replace(' ', '') for col in weatherClass.columns]
weatherClass = weatherClass[['Temperature', 'WindSpeed', 'Precipitation', 'Pressure', 'Season', 'WeatherType']]
weatherClass.tail()

,Temperature,WindSpeed,Precipitation,Pressure,Season,WeatherType
13195,10.0,14.5,71.0,1003.15,Summer,Rainy
13196,-1.0,3.5,23.0,1067.23,Winter,Snowy
13197,30.0,5.5,28.0,1012.69,Autumn,Cloudy
13198,3.0,10.0,94.0,984.27,Winter,Snowy
13199,-5.0,0.0,92.0,1015.37,Autumn,Rainy


In [36]:
from datetime import datetime

def get_season(date):
  month = date.month
  if 3 <= month <= 5:
    return 'Spring'
  elif 6 <= month <= 8:
    return 'Summer'
  elif 9 <= month <= 11:
    return 'Autumn'
  else:
    return 'Winter'

fireClass = cafire.copy()
fireClass['Season'] = fireClass['FireDiscoveryDateTime'].apply(lambda x: get_season(x))
fireClass = fireClass[[]]
fireClass.tail()

,IncidentSize,FireDiscoveryDateTime,IncidentTypeCategory,InitialLatitude,InitialLongitude,POOState,Season
3811,1.80,2024-06-25 21:15:15,WF,35.561729,-120.516991,US-CA,Summer
3812,1.25,2024-06-25 18:04:39,WF,41.759312,-122.490208,US-CA,Summer
3813,11.03,2024-06-25 23:30:19,WF,38.845861,-123.019703,US-CA,Summer
3814,12.00,2024-06-26 02:28:04,WF,36.631882,-119.322600,US-CA,Summer
3815,53.91,2024-06-26 05:00:46,WF,37.732422,-120.596351,US-CA,Summer


In [37]:
fireClass['Temperature'] = fireClass.apply(lambda row: apply_weather_stats(row, 0), axis=1)
fireClass['Precipitation'] = fireClass.apply(lambda row: apply_weather_stats(row, 1), axis=1)
fireClass['WindSpeed'] = fireClass.apply(lambda row: apply_weather_stats(row, 2), axis=1)
fireClass['Pressure'] = fireClass.apply(lambda row: apply_weather_stats(row, 3), axis=1)

fireClass['Precipitation'].fillna(0, inplace=True)
fireClass.dropna(subset=['Temperature'], inplace=True)
fireClass.dropna(subset=['WindSpeed'], inplace=True)
fireClass.dropna(subset=['Pressure'], inplace=True)

fireClass.tail()

,IncidentSize,FireDiscoveryDateTime,IncidentTypeCategory,InitialLatitude,InitialLongitude,POOState,Season,Temperature,Precipitation,WindSpeed,Pressure
3811,1.80,2024-06-25 21:15:15,WF,35.561729,-120.516991,US-CA,Summer,23.6,0.0,12.0,1013.6
3812,1.25,2024-06-25 18:04:39,WF,41.759312,-122.490208,US-CA,Summer,23.3,0.0,15.1,1013.2
3813,11.03,2024-06-25 23:30:19,WF,38.845861,-123.019703,US-CA,Summer,21.3,0.0,10.2,1014.4
3814,12.00,2024-06-26 02:28:04,WF,36.631882,-119.322600,US-CA,Summer,27.4,0.0,14.0,1009.7
3815,53.91,2024-06-26 05:00:46,WF,37.732422,-120.596351,US-CA,Summer,25.5,0.0,16.4,1010.2


In [38]:
fireClass = fireClass[['Temperature', 'WindSpeed', 'Precipitation', 'Pressure', 'Season']]
fireClass['WeatherType'] = 'Fire'
weatherClass = pd.concat([weatherClass, fireClass], ignore_index=True)
weatherClass.tail()

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Temperature,WindSpeed,Precipitation,Pressure,Season,WeatherType
16417,23.6,12.0,0.0,1013.6,Summer,Fire
16418,23.3,15.1,0.0,1013.2,Summer,Fire
16419,21.3,10.2,0.0,1014.4,Summer,Fire
16420,27.4,14.0,0.0,1009.7,Summer,Fire
16421,25.5,16.4,0.0,1010.2,Summer,Fire


In [40]:
#if I want to do size later

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Assume df is your DataFrame and "wildfire_size" is the target variable
X = testdf[['Temperature', 'Precipitation', 'WindSpeed', 'Pressure', 'VegetationIndex', 'MFRI', 'TimeSinceFire']]
y = testdf["IncidentSize"]

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2003)

# Normalize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [42]:
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# Assume df is your DataFrame and "wildfire_size" is the target variable
X = weatherClass[['Temperature', 'Precipitation', 'WindSpeed', 'Pressure', 'Season']]
y = weatherClass["WeatherType"]

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2003)

# Create a ColumnTransformer
ct = ColumnTransformer(
    [('scaler', StandardScaler(), ['Temperature', 'Precipitation', 'WindSpeed', 'Pressure']),
     ('encoder', OneHotEncoder(), ['Season'])],
    remainder='passthrough'  # Include remaining columns as they are
)

# Fit and transform the data
X_train = ct.fit_transform(X_train)
X_test = ct.transform(X_test)

In [46]:
num_classes = weatherClass['WeatherType'].nunique()
print(num_classes)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras import regularizers

model = Sequential()
model.add(Dense(256, activation='relu', input_shape=(X_train.shape[1],), kernel_regularizer=regularizers.l2(0.01)))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(num_classes, activation='softmax'))
# model.add(Dense(1, activation='linear'))  # Output layer for regression

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# model.compile(optimizer='adam', loss='mean_squared_error')


# Ensure your target variable is one-hot encoded for classification
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

from tensorflow.keras.utils import to_categorical
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)


# Train the model on the new data for a few epochs
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)  # Adjust the number of epochs and batch size as needed


5


Epoch 1/50
329/329 ━━━━━━━━━━━━━━━━━━━━ 8s 8ms/step - accuracy: 0.6134 - loss: 4.6739 - val_accuracy: 0.5582 - val_loss: 3.1931
Epoch 2/50
329/329 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.7951 - loss: 2.4781 - val_accuracy: 0.8531 - val_loss: 1.4916
Epoch 3/50
329/329 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.8179 - loss: 1.4014 - val_accuracy: 0.8630 - val_loss: 0.8976
Epoch 4/50
329/329 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.8230 - loss: 0.9348 - val_accuracy: 0.8554 - val_loss: 0.6608
Epoch 5/50
329/329 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.8398 - loss: 0.6955 - val_accuracy: 0.8550 - val_loss: 0.5723
Epoch 6/50
329/329 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.8395 - loss: 0.6285 - val_accuracy: 0.8592 - val_loss: 0.5303
Epoch 7/50
329/329 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.8407 - loss: 0.5980 - val_accuracy: 0.8600 - val_loss: 0.5307
Epoch 8/50
329/329 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.8443 - loss: 0.5714 - val_accuracy: 0.

In [ ]:
#get any location and predict fire or not and probability